In [1]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

In [2]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

## Getting enablement data

In [3]:
# XGS qualifications count at wire center level - manually update date filters every month and at the beginning of the year

query = """
            WITH QD AS (
            SELECT  
                    O.LUID --count distinct LU id's
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    ,case when O.UNI_ECD >= '1-OCT-2022' and O.UNI_ECD < '1-NOV-2022' then 'Y' end as MONTH_CNT_YN
                    ,case when O.UNI_ECD >= '1-JAN-2022' and O.UNI_ECD < '1-JAN-2023' then 'Y' end as YEAR_CNT_YN
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,count(distinct case when MONTH_CNT_YN = 'Y' then LUID end) as monthly_xgs_enabled_cnt
                    ,count(distinct case when YEAR_CNT_YN = 'Y' then LUID end) as ytd_xgs_enabled_cnt
                    ,count(distinct LUID) as cumulative_xgs_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,MONTH_CNT_YN
                    ,YEAR_CNT_YN
            

                                """

query2 = """
            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_POLYGONID
                    ,O.UNI_SERVICETYPE
                    ,O.UNI_STATUS
                    ,O.TECH_TYPE
                    ,O.FSAI
                    ,O.OLT_TYPE
                    ,O.ADDRESS
                    ,O.OLT
                    ,O.UNI_ECD
                    --O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND uni_ecd >= '1-JAN-2022' --'1-JAN-2023' 
            )
            
            SELECT  *
                    
            FROM    QD

                                """

df_ora = pd.read_sql(query, con=conn)
df_ora_details = pd.read_sql(query2, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
%%time

# count of MG enabled LUs at wire center level - manually update date filters
# 10/10/22 - update multigig enablement date to reflect mg turn up date

mg_query = """
           
           WITH MG_DATES AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    ,case when O.UNI_POLYGONID in ('1179498','1179547','1179121','1179499') then to_date('30-SEP-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179687','1179674') then to_date('10-OCT-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179569','1179553','1179688') then to_date('14-OCT-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179864','1180012') then to_date('15-OCT-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179808','1179862','1179836','1179604','1179838','1179887','1180054','1180222','1179157','1179574') then to_date('21-OCT-2022','DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1180028','1180254','1179583') then to_date('27-OCT-2022', 'DD-MON-YYYY')
                          when O.UNI_POLYGONID in ('1179976') then to_date('28-OCT-2022', 'DD-MON-YYYY')
                          else O.UNI_ECD end
                          AS UNI_ECD
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in ('1178832','1178694','1178144',
                                          '1179498','1179547','1179121','1179499',  --Phoenix AZ turned up 9/30/22
                                          '1179687','1179674', -- Golden CO turned up 10/10/22
                                          '1180012', -- portland OR turned up 10/15/22
                                          '1179864', -- denver CO turned up 10/15/22
                                          '1179569','1179553','1179688', -- st paul and mn turned up 10/14/22
                                          '1179808', -- turned up 10/21/22
                                          '1179862','1179836','1179604','1179838','1179887','1180054','1180222', --turned up 10/21/22
                                          '1179157','1179574', -- turned up 10/21/22
                                          '1180028','1180254','1179583', -- turned up 10/27/22
                                          '1179976' -- turned up 10/28/22
                                          ) --add in MG polygon IDs here
                    
            ),
            QD AS (
            
            SELECT  LUID
                    ,STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,ADDRESS
                    ,UNI_SERVICETYPE
                    ,OLT
                    ,TECH_TYPE
                    ,OLT_TYPE
                    ,FSAI
                    ,UNI_POLYGONID
                    ,UNI_STATUS
                    ,UNI_NDSJOBNUMBER
                    ,UNI_FWRKSJOBNUMBER
                    ,case when UNI_ECD >= '1-OCT-2022' and UNI_ECD < '1-NOV-2022' then 'Y' end as MONTH_CNT_YN
                    ,case when UNI_ECD >= '1-JAN-2022' and UNI_ECD < '1-JAN-2023' then 'Y' end as YEAR_CNT_YN
                    
            FROM MG_DATES
            
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,count(distinct case when MONTH_CNT_YN = 'Y' then LUID end) as monthly_mg_enabled_cnt
                    ,count(distinct case when YEAR_CNT_YN = 'Y' then LUID end) as ytd_mg_enabled_cnt
                    ,count(distinct LUID) as cumulative_mg_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,MONTH_CNT_YN
                    ,YEAR_CNT_YN


"""

mg_lu_ora = pd.read_sql(mg_query, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 0 ns
Wall time: 707 ms


In [5]:
mg_lu_ora.groupby(['STATE'], as_index = False).sum()

,STATE,MONTHLY_MG_ENABLED_CNT,YTD_MG_ENABLED_CNT,CUMULATIVE_MG_ENABLED_CNT
0,AZ,0,1062,1062
1,CO,1015,1211,1211
2,MN,5411,5747,5747
3,OR,74,74,74
4,WA,0,342,342


In [6]:
# sum queries at wire center level - add mg flag according to mg enablement or core network status
xgs_enablement_wc = df_ora.groupby(['STATE','PRIMARY_WIRE_CENTER_ID'], as_index = False).sum()
mg_enablement_wc = mg_lu_ora.groupby(['STATE','PRIMARY_WIRE_CENTER_ID'], as_index = False).sum()

In [7]:
# merge xgs and mg enablement
luqual_enablement_df = xgs_enablement_wc.merge(mg_enablement_wc, how = 'outer', on = ['STATE','PRIMARY_WIRE_CENTER_ID']).fillna(0)
luqual_enablement_df_save = luqual_enablement_df.copy() #save copy

In [8]:
luqual_enablement_df

,STATE,PRIMARY_WIRE_CENTER_ID,MONTHLY_XGS_ENABLED_CNT,YTD_XGS_ENABLED_CNT,CUMULATIVE_XGS_ENABLED_CNT,MONTHLY_MG_ENABLED_CNT,YTD_MG_ENABLED_CNT,CUMULATIVE_MG_ENABLED_CNT
0,AZ,PHNXAZNE,0,212,212,0.0,212.0,212.0
1,AZ,PHNXAZSO,306,1156,1156,0.0,850.0,850.0
2,AZ,SPRSAZEA,0,295,295,0.0,0.0,0.0
3,AZ,SPRSAZMA,0,259,259,0.0,0.0,0.0
4,AZ,TCSNAZCR,0,400,400,0.0,0.0,0.0
5,AZ,TCSNAZFW,0,95,95,0.0,0.0,0.0
6,AZ,TCSNAZMA,2056,6047,6047,0.0,0.0,0.0
7,AZ,TCSNAZRN,298,406,406,0.0,0.0,0.0
8,AZ,TCSNAZSW,163,163,163,0.0,0.0,0.0
9,CO,ARVDCOMA,0,196,196,0.0,196.0,196.0


In [9]:
# check data
scorecard_view = luqual_enablement_df[['STATE','CUMULATIVE_XGS_ENABLED_CNT','CUMULATIVE_MG_ENABLED_CNT','MONTHLY_XGS_ENABLED_CNT','MONTHLY_MG_ENABLED_CNT']].groupby(['STATE'], as_index = False).sum()

In [10]:
# import salesforce summary to get active subs
sf_df_xgs_sum = pd.read_excel(r'SalesforceReportExport/Salesforce_Summary.xlsx',sheet_name = 'State_Summary', skiprows=[0,1])
sf_df_mg_sum = pd.read_excel(r'SalesforceReportExport/Salesforce_Summary.xlsx',sheet_name = 'MultiGig_State_Summary', skiprows=[0,1]) 

In [11]:
sf_df_xgs_sum = sf_df_xgs_sum[['STATE','Total Active Subscribers']].rename(columns = {'Total Active Subscribers':'Total_XGS_Active_subs'})
sf_df_mg_sum = sf_df_mg_sum[['STATE','Total Active Subscribers']].rename(columns = {'Total Active Subscribers':'Total_MG_Active_subs'})
salesforce_subscribers_summary = sf_df_xgs_sum.merge(sf_df_mg_sum, how = 'left', on = 'STATE')
salesforce_subscribers_summary = salesforce_subscribers_summary.loc[salesforce_subscribers_summary['STATE']!=0,:]

In [12]:
scorecard_view = scorecard_view.merge(salesforce_subscribers_summary, on = 'STATE', how = 'left')

In [13]:
scorecard_view.loc['Total'] = scorecard_view.sum(numeric_only=True)

In [14]:
scorecard_view.to_excel('Path_To_Premier_Summary.xlsx')